#  Data analysis for Lexical Richness

In [1]:
#importing libraries
import os
import nltk
import numpy as np
from nltk.tokenize import RegexpTokenizer
import pandas as pd

# Functions

In [ ]:
#binary search for strings
def binarySearchArr(arr, x):
    l = 0
    r = len(arr)
    while (l <= r):
        m = l + ((r - l) // 2)
 
        res = (x == arr[m])
        # Check if x is present at mid
        if (res == True):
            return m
 
        # If x greater, ignore left half
    #problem is here
        elif (res == False):
            if (x>arr[m]):
                l = m+1
            elif (x<arr[m]):
                r=m-1
        # If x is smaller, ignore right half
    return -1

In [ ]:
#This function removes all of the rows that contain a string in the passed column
def rmStr (df, col):
    row_index=0
    row_ind=[]
    for i in df[col]:
        try:
            int(i)
        except:
            row_ind.append(row_index)
        row_index+=1
    return df.drop(labels=row_ind,axis=0)

In [ ]:
# reference code from previous research, tokinizes essays
# NEEDS UPDATING TO FIT CSV FILES

#function that does this
def calculate_tokens(file):
    essay = ''

    with open(file, 'r') as f: #used bc it's a file
        essay = f.read().lower()
        
    tokenizer = RegexpTokenizer(r'[0-9a-zA-Z_\-]+')
    tokenized = tokenizer.tokenize(essay)
    unique = np.unique(tokenized)
    
    return {
        'total': len(tokenized),
        'unique': len(unique)
    }


# Adding DFs

In [8]:
loose=pd.read_csv('original-sheets\Responses-Loose.csv')
student_info=pd.read_csv('corpus-files\student_information.csv')
course=pd.read_csv('corpus-files\course.csv')
answer_df=pd.read_csv('answer.csv',index_col = 'answer_id', low_memory=False)

In [3]:
course_=course.drop(columns=['class_id','semester','section'])

student=student_info.drop(columns=['birth_year','language_used_at_home','non_native_language_1','yrs_of_study_lang1',
                                  'study_in_classroom_lang1','ways_of_study_lang1','non_native_language_2','study_in_classroom_lang2',
                                  'ways_of_study_lang2','non_native_language_3','yrs_of_study_lang3','study_in_classroom_lang3',
                                  'ways_of_study_lang3','yrs_of_english_learning','yrs_in_english_environment','yrs_of_study_lang2'])
answer_=answer_df[['question_id','anon_id','course_id','version','text_len','text','tokens']].reset_index()

In [9]:
loose.head()
#use text3 for word count?
#use text3_len for number of tokes, but doesn't actually get the tokens

,3Langs,Item,answer_id,anon_id,L1,gender,course_id,level_id,class_id,question_id,version,text1_len,text2_len,text3_len,text1,"text2 (line breaks/extra spaces removed, spaces added to reach 60)",text3 (edits made to fix word counts),Judgement,Notes
0,Y,C12,141.0,aj8,Chinese,Male,118.0,5.0,w,17.0,1.0,121.0,114.0,114.0,"\n\n\n Today, I am going to describe one of my...","Today, I am going to describe one of my classm...","Today, I am going to describe one of my classm...",1.0,NaN
1,Y,C13,143.0,az8,Chinese,Female,118.0,5.0,w,17.0,1.0,96.0,95.0,95.0,My niece is 3 years old who is my younger brot...,My niece is 3 years old who is my younger brot...,My niece is 3 years old who is my younger brot...,1.0,NaN
2,Y,K9,133.0,az2,Korean,Male,118.0,5.0,w,17.0,1.0,130.0,128.0,128.0,"When I was in Germany, I met a friend who was ...","When I was in Germany, I met a friend who was ...","When I was in Germany, I met a friend who was ...",1.0,NaN
3,Y,K11,135.0,at8,Korean,Female,118.0,5.0,w,17.0,3.0,104.0,105.0,105.0,my friend is ANON_NAME_0. she is a my ELI frie...,my friend is ANON_NAME_0. she is a my ELI frie...,my friend is ANON_NAME_0. she is a my ELI frie...,1.0,NaN
4,Y,K20,188.0,eh9,Korean,Male,118.0,5.0,w,17.0,1.0,98.0,97.0,97.0,Younghun is my best friend. His facial appeara...,Younghun is my best friend. His facial appeara...,Younghun is my best friend. His facial appeara...,1.0,NaN


In [5]:
#Cleaning the answer df
col='course_id'
answer=rmStr(answer_, col) #removes strings from answer_.course_id 

In [6]:
# Merging the datasets

merge_ans=answer[['anon_id','course_id']] #sets answer df up for merging 
student_merge=student.drop(columns=['course_history']) #set student df up for merging

student_ans=student_merge.merge(answer,on='anon_id').astype({'course_id':'int64'}) #merges student and answers

stu_ans_crs=student_ans.merge(course_, on='course_id') #merges the student-answers df with course df

In [7]:
stu_ans_crs

,anon_id,gender,native_language,age,answer_id,question_id,course_id,version,text_len,text,tokens,level_id
0,do6,Female,Russian,31.0,150,4,117,1,299,Some people prefer eat out and some like doing...,"['Some', 'people', 'prefer', 'eat', 'out', 'an...",5
1,do6,Female,Russian,31.0,1221,97,117,1,288,My opinion is that a person does need educatio...,"['My', 'opinion', 'is', 'that', 'a', 'person',...",5
2,do6,Female,Russian,31.0,1957,189,117,1,321,There are two national rooms in the Cathedral ...,"['There', 'are', 'two', 'national', 'rooms', '...",5
3,do6,Female,Russian,31.0,2164,190,117,1,464,There are two nation rooms in the Cathedral of...,"['There', 'are', 'two', 'nation', 'rooms', 'in...",5
4,bv5,Male,Arabic,21.0,151,4,117,1,315,"""Not all learning takes place in the classroom...","['``', 'Not', 'all', 'learning', 'takes', 'pla...",5
...,...,...,...,...,...,...,...,...,...,...,...,...
46225,cy7,Female,Korean,NaN,47682,6066,1034,1,10,1. emphasis\n2. appropriate\n3. requires\n4. a...,"['1', '.', 'emphasis', '2', '.', 'appropriate'...",3
46226,fp7,Female,Turkish,NaN,47823,6066,1034,1,10,1.emphasis \n2.appropriate\n3.requires\n4.anal...,"['1', '.', 'emphasis', '2', '.', 'appropriate'...",3
46227,fq6,Male,Chinese,NaN,47824,6066,1034,1,10,1.emphasis\n2.appropriate\n3.requires\n4.analy...,"['1', '.', 'emphasis', '2', '.', 'appropriate'...",3
46228,di6,Male,Chinese,NaN,47787,6066,1034,1,10,1 emphasis\n2 normal\n3 requires\n4 analyze\n5...,"['1', 'emphasis', '2', 'normal', '3', 'require...",3


# Tokinizing Essays for Loose

In [ ]:
# reference code from previous research

#function that does this
def calculate_tokens(file):
    essay = ''

    with open(file, 'r') as f:
        essay = f.read().lower()
        
    tokenizer = RegexpTokenizer(r'[0-9a-zA-Z_\-]+')
    tokenized = tokenizer.tokenize(essay)
    unique = np.unique(tokenized)
    
    return {
        'total': len(tokenized),
        'unique': len(unique)
    }


# don't show this code to anyone else, i'm a bad boy
directories = ['limited', 'loose', 'strictest']
tokenize_results = { 'filename': [], 'total_tokens': [], 'unique_tokens': [] }

for directory in directories:
    for file in os.listdir(f'./data/in/{directory}'):
        results = calculate_tokens(f'./data/in/{directory}/{file}')
        tokenize_results['filename'].append(file)
        tokenize_results['total_tokens'].append(results['total'])
        tokenize_results['unique_tokens'].append(results['unique'])

pd.DataFrame(data=tokenize_results)